# DeepCpG basics

This tutorial describes how to create the input data for DeepCpG, train models, and use the trained models for imputation.

## Variables

We first initialize some variables that will be used throughout the tutorial.

In [1]:
data_dir="../data"
cpg_dir="${data_dir}/cpg"
dna_dir="$data_dir/dna/mm10"

## Creating DeepCpG data files

We first store the known CpG methylation states of each cell into a tab delimted file with the following columns:
* Chromosome (without chr)
* Position of the CpG site on the chromosome
* Binary methylation state of the CpG sites (0=unmethylation, 1=methylated)

CpG sites with a methylation rate between zero and one should be binarized by rounding. Filenames should correspond to cell names. 

For this tutorial we are using a subset of serum mouse embryonic stem cells from *Smallwood et al. (2014)*:

In [2]:
ls $cpg_dir

BS27_1_SER.tsv BS27_3_SER.tsv BS27_5_SER.tsv BS27_6_SER.tsv BS27_8_SER.tsv


We can have a look at the methylation profile of cell 'BS27_1_SER':

In [3]:
head "$cpg_dir/BS27_1_SER.tsv"

1	3000827	1.0
1	3001007	1.0
1	3001018	1.0
1	3001277	1.0
1	3001629	1.0
1	3003226	1.0
1	3003339	1.0
1	3003379	1.0
1	3006416	1.0
1	3007580	1.0


Since we are dealing with mouse cells, we are using the mm10 (GRCm38) mouse genome build:

In [4]:
ls $dna_dir

Mus_musculus.GRCm38.dna.chromosome.1.fa.gz
Mus_musculus.GRCm38.dna.chromosome.10.fa.gz
Mus_musculus.GRCm38.dna.chromosome.11.fa.gz
Mus_musculus.GRCm38.dna.chromosome.12.fa.gz
Mus_musculus.GRCm38.dna.chromosome.13.fa.gz
Mus_musculus.GRCm38.dna.chromosome.14.fa.gz
Mus_musculus.GRCm38.dna.chromosome.15.fa.gz
Mus_musculus.GRCm38.dna.chromosome.16.fa.gz
Mus_musculus.GRCm38.dna.chromosome.17.fa.gz
Mus_musculus.GRCm38.dna.chromosome.18.fa.gz
Mus_musculus.GRCm38.dna.chromosome.19.fa.gz
Mus_musculus.GRCm38.dna.chromosome.2.fa.gz
Mus_musculus.GRCm38.dna.chromosome.3.fa.gz
Mus_musculus.GRCm38.dna.chromosome.4.fa.gz
Mus_musculus.GRCm38.dna.chromosome.5.fa.gz
Mus_musculus.GRCm38.dna.chromosome.6.fa.gz
Mus_musculus.GRCm38.dna.chromosome.7.fa.gz
Mus_musculus.GRCm38.dna.chromosome.8.fa.gz
Mus_musculus.GRCm38.dna.chromosome.9.fa.gz
Mus_musculus.GRCm38.dna.chromosome.MT.fa.gz
Mus_musculus.GRCm38.dna.chromosome.X.fa.gz
Mus_musculus.GRCm38.dna.chromosome.Y.fa.gz


These files were downloaded by `setup.py`. Other genomes, e.g. human genome hg38, can be downloaded, for example, with the following command:

```bash
wget ftp://ftp.ensembl.org/pub/release-86/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.*.fa.gz
```

Now we can run `dcpg_data.py` to create the input data for DeepCpG. For demostration purposes, we only consider chromosome 19:

In [5]:
dcpg_data="./data"
cmd="dcpg_data.py
    --cpg_profiles $cpg_dir/*.tsv
    --dna_files $dna_dir
    --out_dir $dcpg_data
    --cpg_wlen 50
    --dna_wlen 1001
    --chromo 19
    "
eval $cmd

INFO (2017-01-29 19:08:55,693): Reading single-cell profiles ...
INFO (2017-01-29 19:09:36,636): 306801 samples
INFO (2017-01-29 19:09:36,647): --------------------------------------------------------------------------------
INFO (2017-01-29 19:09:36,647): Chromosome 19 ...
INFO (2017-01-29 19:09:37,147): 306801 / 306801 (100.0%) sites matched minimum coverage filter
INFO (2017-01-29 19:09:38,357): Chunk 	1 / 10
INFO (2017-01-29 19:09:38,393): Extracting DNA sequence windows ...
INFO (2017-01-29 19:09:43,890): Extracting CpG neighbors ...
INFO (2017-01-29 19:09:47,434): Chunk 	2 / 10
INFO (2017-01-29 19:09:47,467): Extracting DNA sequence windows ...
INFO (2017-01-29 19:09:53,141): Extracting CpG neighbors ...
INFO (2017-01-29 19:09:56,670): Chunk 	3 / 10
INFO (2017-01-29 19:09:56,706): Extracting DNA sequence windows ...
INFO (2017-01-29 19:10:02,381): Extracting CpG neighbors ...
INFO (2017-01-29 19:10:05,979): Chunk 	4 / 10
INFO (2017-01-29 19:10:06,015): Extracting DNA sequence win

For each CpG site that is observed in at least one cell, this command extracts the 50 neighboring CpG sites (25 to the left and 25 to the right), and the 1000 bp long DNA sequence window centered on the CpG site. The command creates multiple HDF5 files with name `cX_FROM_TO.h5`, where `X` is the chromosome, and `FROM` and `TO` the index of CpG sites stored in the file:

In [6]:
ls $dcpg_data

c19_000000-032768.h5 c19_131072-163840.h5 c19_262144-294912.h5
c19_032768-065536.h5 c19_163840-196608.h5 c19_294912-306801.h5
c19_065536-098304.h5 c19_196608-229376.h5 pos.txt
c19_098304-131072.h5 c19_229376-262144.h5


## Model training 

We can now train models on the created data files. 

First, we train a model that only uses the neighboring methylation states of all cells, denoted as *CpG module* in the publication. For demonstration purposes, we only train for one epoch on 1000 CpG sites. In practice, one would train on all data until learning converges, if possible on one or more GPUs. 

In [7]:
models_dir="./models"
mkdir -p $models_dir

In [8]:
cmd="dcpg_train.py
    $dcpg_data/c*.h5
    --val_files $dcpg_data/c*.h5
    --cpg_model RnnL1
    --out_dir $models_dir/cpg
    --nb_epoch 1
    --nb_train_sample 1000
    --nb_val_sample 1000
    "
eval $cmd

Using TensorFlow backend.
INFO (2017-01-29 19:11:19,183): Building model ...
Replicate names:
BS27_1_SER, BS27_3_SER, BS27_5_SER, BS27_6_SER, BS27_8_SER

INFO (2017-01-29 19:11:19,191): Building CpG model ...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cpg/state/BS27_1_SER--BS27_3_SER (None, 5, 50)         0                                            
____________________________________________________________________________________________________
cpg/dist/BS27_1_SER--BS27_3_SER- (None, 5, 50)         0                                            
____________________________________________________________________________________________________
cpg/merge_1 (Merge)              (None, 5, 100)        0           cpg/state/BS27_1_SER--BS27_3_SER-
                                                                   cpg/dist/BS27_1_S

Altough the model only uses neighboring CpG sites, it is already quite accurate in practice. To also make use of the DNA sequence, we train a `DNA module`:

In [9]:
cmd="dcpg_train.py
    $dcpg_data/c*.h5
    --val_files $dcpg_data/c*.h5
    --dna_model CnnL2h128
    --out_dir $models_dir/dna
    --nb_epoch 1
    --nb_train_sample 1000
    --nb_val_sample 1000
    "
eval $cmd

Using TensorFlow backend.
INFO (2017-01-29 19:11:34,497): Building model ...
INFO (2017-01-29 19:11:34,500): Building DNA model ...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dna (InputLayer)                 (None, 1001, 4)       0                                            
____________________________________________________________________________________________________
dna/convolution1d_1 (Convolution (None, 991, 128)      5760        dna[0][0]                        
____________________________________________________________________________________________________
dna/activation_1 (Activation)    (None, 991, 128)      0           dna/convolution1d_1[0][0]        
____________________________________________________________________________________________________
dna/maxpooling1d_1 (MaxPooling1D (None, 247, 128)      0    

Finally, we are combining both models by training a *joint module* without training the *CpG* and *DNA* module:

In [10]:
cmd="dcpg_train.py
    $dcpg_data/c*.h5
    --val_files $dcpg_data/c*.h5
    --dna_model $models_dir/dna
    --cpg_model $models_dir/cpg
    --joint_model JointL2h512
    --train_models joint
    --out_dir $models_dir/joint
    --nb_epoch 1
    --nb_train_sample 1000
    --nb_val_sample 1000
    "
eval $cmd

Using TensorFlow backend.
INFO (2017-01-29 19:12:04,308): Building model ...
INFO (2017-01-29 19:12:04,310): Loading existing DNA model ...
INFO (2017-01-29 19:12:04,311): Using model files ./models/dna/model.json ./models/dna/model_weights_val.h5
Replicate names:
BS27_1_SER, BS27_3_SER, BS27_5_SER, BS27_6_SER, BS27_8_SER

INFO (2017-01-29 19:12:04,619): Loading existing CpG model ...
INFO (2017-01-29 19:12:04,619): Using model files ./models/cpg/model.json ./models/cpg/model_weights_val.h5
INFO (2017-01-29 19:12:05,375): Joining models ...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dna (InputLayer)                 (None, 1001, 4)       0                                            
____________________________________________________________________________________________________
dna/convolution1d_1 (Convolution (None, 991, 128)

## Model evaluation 

We are now using `dcpg_eval.py` to finally impute the missing methylation states and to evaluate prediction performance on the partially observed methylation states. We will use the trained joint module, but could of course also evaluate the CpG or DNA module only.

In [11]:
eval_dir="./eval"
mkdir -p $eval_dir

cmd="dcpg_eval.py
    $dcpg_data/c*.h5
    --model_files $models_dir/joint
    --out_data $eval_dir/data.h5
    --out_report $eval_dir/report.tsv
    --nb_sample 1000
    "
eval $cmd

Using TensorFlow backend.
INFO (2017-01-29 19:12:28,866): Loading model ...
INFO (2017-01-29 19:12:29,829): Loading data ...
INFO (2017-01-29 19:12:29,830): Predicting ...
INFO (2017-01-29 19:12:29,865):  128/1000 (12.8%)
INFO (2017-01-29 19:12:30,341):  256/1000 (25.6%)
INFO (2017-01-29 19:12:30,794):  384/1000 (38.4%)
INFO (2017-01-29 19:12:31,262):  512/1000 (51.2%)
INFO (2017-01-29 19:12:31,725):  640/1000 (64.0%)
INFO (2017-01-29 19:12:32,157):  768/1000 (76.8%)
INFO (2017-01-29 19:12:32,588):  896/1000 (89.6%)
INFO (2017-01-29 19:12:33,032): 1000/1000 (100.0%)
/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
           output       auc       acc       tpr  tnr        f1       mcc      n
4  cpg/BS27_8_SER  0.663961  0.161765  0.002915  1.0  0.005814  0.021578  4

The imputed methylation profiles of all cells are stored in `data.h5`, and performance metrics in `report.tsv`

In [12]:
cat eval/report.tsv

metric	output	value
acc	cpg/BS27_3_SER	0.08275862068965517
acc	cpg/BS27_1_SER	0.14285714285714285
acc	cpg/BS27_8_SER	0.16176470588235295
acc	cpg/BS27_5_SER	0.30414746543778803
acc	cpg/BS27_6_SER	0.3172690763052209
auc	cpg/BS27_1_SER	0.34103954081632654
auc	cpg/BS27_6_SER	0.4475800446760983
auc	cpg/BS27_3_SER	0.4895050125313284
auc	cpg/BS27_5_SER	0.5454294601645595
auc	cpg/BS27_8_SER	0.6639605292666516
f1	cpg/BS27_6_SER	0.0
f1	cpg/BS27_5_SER	0.0
f1	cpg/BS27_1_SER	0.0
f1	cpg/BS27_3_SER	0.0
f1	cpg/BS27_8_SER	0.005813953488372092
mcc	cpg/BS27_6_SER	0.0
mcc	cpg/BS27_5_SER	0.0
mcc	cpg/BS27_1_SER	0.0
mcc	cpg/BS27_3_SER	0.0
mcc	cpg/BS27_8_SER	0.02157806086076016
n	cpg/BS27_6_SER	249.0
n	cpg/BS27_3_SER	290.0
n	cpg/BS27_1_SER	392.0
n	cpg/BS27_8_SER	408.0
n	cpg/BS27_5_SER	434.0
tnr	cpg/BS27_6_SER	1.0
tnr	cpg/BS27_5_SER	1.0
tnr	cpg/BS27_1_SER	1.0
tnr	cpg/BS27_8_SER	1.0
tnr	cpg/BS27_3_SER	1.0
tpr	cpg/BS27_6_SER	0.0
tpr	cpg/BS27_5_SER	0.0
tpr	cpg/BS27_1_SER	0.0
tpr	cpg/BS27_3_SER	0.0
tpr	cpg/BS27_8_

In [13]:
h5ls -r eval/data.h5

/                        Group
/chromo                  Dataset {1000}
/outputs                 Group
/outputs/cpg             Group
/outputs/cpg/BS27_1_SER  Dataset {1000}
/outputs/cpg/BS27_3_SER  Dataset {1000}
/outputs/cpg/BS27_5_SER  Dataset {1000}
/outputs/cpg/BS27_6_SER  Dataset {1000}
/outputs/cpg/BS27_8_SER  Dataset {1000}
/pos                     Dataset {1000}
/preds                   Group
/preds/cpg               Group
/preds/cpg/BS27_1_SER    Dataset {1000}
/preds/cpg/BS27_3_SER    Dataset {1000}
/preds/cpg/BS27_5_SER    Dataset {1000}
/preds/cpg/BS27_6_SER    Dataset {1000}
/preds/cpg/BS27_8_SER    Dataset {1000}
